In [ ]:
%pip install -e .
%pip install -r requirements.txt
%pip install nest_asyncio

In [1]:
import nest_asyncio

# Allow backtests to run in notebook
nest_asyncio.apply()

# Increase width of console output for browsers
from IPython.display import HTML, display


display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [1]:
import pandas as pd


from lazyft import backtest, paths, remote
from lazyft.backtest.runner import BacktestMultiRunner
from lazyft.config import Config
from lazyft.reports import get_backtest_repo, get_hyperopt_repo

ImportError: cannot import name 'PerformanceBase' from partially initialized module 'lazyft.models' (most likely due to a circular import) (/home/raphael/PycharmProjects/lazyft_lib/lazyft/models/__init__.py)

# Prepare to backtest

In [ ]:
from lazyft.command_parameters import BacktestParameters

b_timerange = "20220301-20220401"
# b_days = 30

b_conf = Config('config.json')

b_params = BacktestParameters(
    timerange=b_timerange,
    interval='1h',
    config_path=b_conf,
    # days=b_days,
    # secrets_config=priv_bin_us,
    # pairs=['PYR/USDT'],
    stake_amount='unlimited',
    # timeframe_detail='5m',
    starting_balance=100,
    max_open_trades=3,
    download_data=True,
    tag='',
)
print(len(b_params.pairs), 'pairs')
# b_params.max_open_trades = len(b_params.pairs)
# print('strategy:', b_params.strategies[0])
print('max open trades:', b_params.max_open_trades)
print('stake amount:', b_params.stake_amount)
print('timeframe:', b_params.timerange)
print('starting balance:', b_params.starting_balance)
print('exchange:', b_params.config['exchange']['name'])
print('timeframe:', b_params.interval)
if b_params.ensemble:
    print('ensemble:', b_params.ensemble)

## Run Backtest on Gumbo1

In [ ]:
b_runner = b_params.run('Gumbo1', load_from_hash=False)
b_runner.save()

## Show backtest

In [ ]:
print(b_runner.report.report_text)

In [ ]:
b_runner.report.df

In [ ]:
b_runner.report.plot_weekly()

# Prepare to Hyperopt Gumbo1

In [ ]:
from lazyft.command_parameters import HyperoptParameters

h_days = 365
h_conf = Config('config.json')
starting_balance = 100
h_timerange = "20220101-20220331"
# VRA/USDT HTR/USDT ARX/USDT
params = HyperoptParameters(
    epochs=50,
    config_path=h_conf,
    #     secrets_config=str(priv_us),
    # days=h_days,
    timerange=h_timerange,
    spaces="default",
    # loss='CalmarHyperOptLoss',
    loss='SortinoHyperOptLoss',
    interval="1h",
    # min_trades=int(days * 0.67 * 0.85),
    min_trades=10,
    #     pairs=pairlist.load_pairlist_from_id(1),
    starting_balance=starting_balance,
    max_open_trades=3,
    stake_amount="unlimited",
    jobs=-2,
    download_data=True,
    # custom_spaces='atr_roi atr_stoploss',
    # custom_settings={
    #     'use_custom_stoploss': False,
    # },
    tag="",
    # seed=62581,
    #     ensemble=ensemble
)
# params.tag = params.tag + ',' + params.pairs[0]
# params.min_trades = len(params.pairs)
print("\ntimerange: ", params.timerange,
      "\ndays: ", round(params.days * 0.667),
      '\nspaces:', params.spaces,
      '\nloss function:', params.loss,
      '\nepochs:', params.epochs,
      '\nmin trades:', params.min_trades,
      '\ntag:', params.tag,
      '\ninterval:', params.interval,
      '\nnum pairs:', len(params.pairs))
# QuickTools.refresh_pairlist(us_conf, 100)

## Run Hyperopt

In [ ]:
h_runner = params.run('Strategy', background=True)

In [ ]:
print(get_hyperopt_repo().filter_by_id([2]).df().to_markdown())

## Show

In [ ]:
h_runner.report.df

## Save

In [ ]:
h_runner.save()

# Backtest Gumbo1 Using Hyperopted Parameters

In [ ]:
bh_runner = b_params.run('Gumbo1-1', load_from_hash=False)

In [ ]:
bh_runner.save()

## Show

In [ ]:
print(bh_runner.report.report_text)

In [ ]:
bh_runner.report.plot() 

# Browse Repo

In [ ]:
get_backtest_repo()

In [ ]:
get_backtest_repo().df()

In [ ]:
get_hyperopt_repo().first().backtest_data.keys()